In [21]:
from pydantic import BaseModel, EmailStr
from datetime import date
import datetime

class Post(BaseModel) :
    post_id: int
    title: str
    content: str
    publish_date: datetime.datetime
    owner_username: str
    # class Config:
    #     orm_mode = True
    #     ignore_extra = True

d = {
    # '_sa_instance_state': "<sqlalchemy.orm.state.InstanceState object at 0x105135f70>",
    'content': 'hello world',
    'owner_username': 'user1',
    'post_id': 1,
    'publish_date': datetime.datetime(2023, 8, 17, 21, 20),
    'title': 'first post'
 }

In [22]:
Post(**d)


Post(post_id=1, title='first post', content='hello world', publish_date=datetime.datetime(2023, 8, 17, 21, 20), owner_username='user1')

In [1]:
from pydantic import BaseModel, EmailStr, ConfigDict
from typing import Optional, List
import datetime

from sqlalchemy import VARCHAR, Column, TEXT, Integer, ForeignKey
from sqlalchemy.orm import relationship

from sqlalchemy.sql.sqltypes import TIMESTAMP
from sqlalchemy.sql.expression import text

from database import Base

class PostORM(Base) :
    __tablename__ = "posts"
    post_id = Column(Integer, primary_key=True)
    title= Column(VARCHAR(50), nullable=True)
    content = Column(TEXT, nullable=True)
    owner_username = Column(VARCHAR(50), ForeignKey("users.username", ondelete="cascade"), nullable=False)


In [11]:

class Post(BaseModel) :
    model_config = ConfigDict(from_attributes=True)
    post_id: int
    title: str
    content: str
    owner_username: str

class PostLike(BaseModel): 
    model_config = ConfigDict(from_attributes=True)
    post: Post  # pydantic Post
    likes: int
    # post: List[Post, int]

In [3]:
post_orm = PostORM(
    post_id=1,
    title="title",
    content="content",
    owner_username="username"
)
post_orm2 = PostORM(
    post_id=1,
    title="title",
    content="content",
    owner_username="username"
)

In [4]:
post_orm

In [5]:
Post.model_validate(post_orm)


Post(post_id=1, title='title', content='content', owner_username='username')

In [13]:
result = [(post_orm, 2), (post_orm2, 1)]
result

[(<__main__.PostORM at 0x1049ec750>, 2),
 (<__main__.PostORM at 0x10784a710>, 1)]

In [16]:
list(map(lambda pl: PostLike(post=Post.model_validate(pl[0]), likes=pl[1]), result))

[PostLike(post=Post(post_id=1, title='title', content='content', owner_username='username'), likes=2),
 PostLike(post=Post(post_id=1, title='title', content='content', owner_username='username'), likes=1)]

In [9]:
post_like = PostLike(
    post=Post.model_validate(post_orm),
    likes=result[1]
)

In [10]:
post_like

PostLike(post=Post(post_id=1, title='title', content='content', owner_username='username'), likes=2)